## Iteration 8

This iteration will use the build number, the platform version, the bitwise breakdown of AVProductSignatures and all the values from the maybe category.

In [22]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier


In [23]:
file="50ksample.csv"

In [24]:
data_df=pd.read_csv(file)
data_df.head()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,1514734,2b7b688e4a4315bc4f21f6849d03ab66,win8defender,1.1.15200.1,4.11.15063.0,1.275.828.0,0,7.0,0,NaN,...,33968.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
1,2926031,53f3fd511ba64acaec4e0c8559143301,win8defender,1.1.15200.1,4.18.1807.18075,1.275.727.0,0,7.0,0,NaN,...,19951.0,0,0.0,0.0,0,0,0.0,1.0,5.0,0
2,7189617,ce48198690d1e4b437ffb60bfcb9e8df,win8defender,1.1.15100.1,4.18.1807.18075,1.273.516.0,0,7.0,0,NaN,...,33066.0,0,NaN,0.0,0,0,0.0,1.0,13.0,1
3,6264912,b3cfee470ee30c729d3a33f0017a8b51,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,...,12303.0,1,NaN,0.0,0,0,0.0,0.0,3.0,0
4,3321923,5f4f1fa2419439ff865d42a2a06ac8b5,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1486.0,0,7.0,0,NaN,...,4357.0,0,NaN,0.0,0,1,0.0,0.0,3.0,1


In [25]:
data_df=data_df[["EngineVersion","AVProductStatesIdentifier","Processor","SmartScreen","Census_MDC2FormFactor","Platform","OsBuild", 
                 "Census_IsAlwaysOnAlwaysConnectedCapable","Wdft_IsGamer",
                 "RtpStateBitfield", "IsSxsPassiveMode" , "Census_HasOpticalDiskDrive", "Census_IsVirtualDevice", "HasDetections"]]

data_df.head()


,EngineVersion,AVProductStatesIdentifier,Processor,SmartScreen,Census_MDC2FormFactor,Platform,OsBuild,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,RtpStateBitfield,IsSxsPassiveMode,Census_HasOpticalDiskDrive,Census_IsVirtualDevice,HasDetections
0,1.1.15200.1,53447.0,x64,NaN,Desktop,windows10,15063,0.0,0.0,7.0,0,0,0.0,0
1,1.1.15200.1,53447.0,x64,RequireAdmin,Desktop,windows10,16299,0.0,1.0,7.0,0,0,0.0,0
2,1.1.15100.1,53447.0,x64,NaN,Notebook,windows10,15063,0.0,1.0,7.0,0,0,0.0,1
3,1.1.14600.4,47238.0,x64,NaN,Notebook,windows10,17134,0.0,0.0,7.0,0,0,0.0,0
4,1.1.15200.1,53447.0,x64,NaN,Desktop,windows10,17134,0.0,0.0,7.0,0,0,0.0,1


Nulls in AVProductStatesIdentifier are set to 0

Browser Identtifier needs to be filled wiht 0
SmartScreen: clean up off and Off, on and On, “&#x02” and “&#x01" will have own columns., null should go to off bucket
Census_IsAlwaysConnectedCapable: replace with 0
Wdft_IsGamer: replace with 0
RtpStateBitfield: replace with 0
Census_IsVirtualDevice: replace with 0




In [26]:
data_df[["Census_IsAlwaysOnAlwaysConnectedCapable","Wdft_IsGamer", "RtpStateBitfield", "Census_IsVirtualDevice"]].fillna(value=0, inplace=True)
data_df["SmartScreen"].fillna(value="off", inplace=True)
data_df["SmartScreen"].replace("Off", "off", inplace=True)
data_df["SmartScreen"].replace("On", "on", inplace=True)
data_df["SmartScreen"].fillna(value="off", inplace=True)
data_df["AVProductStatesIdentifier"].fillna(value="0", inplace=True)

## This drops any remaining nulls 
data_df.dropna(how='any',inplace=True)


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [27]:
# Convert the float in the AVProductStatesIdentifier feature to a binary
data_df['AVProductStatesIdentifier'] = data_df.AVProductStatesIdentifier.apply(lambda x: format(int(x), '04b'))
# Convert the Binary to a string
data_df['AVProductStatesIdentifier']=data_df['AVProductStatesIdentifier'].astype(str)
# Add Leading 0's to the sting
data_df["AVProductStatesIdentifier"]=data_df["AVProductStatesIdentifier"].str.zfill(17)
data_df.head()

In [31]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47779 entries, 0 to 49999
Data columns (total 14 columns):
EngineVersion                              47779 non-null object
AVProductStatesIdentifier                  47779 non-null object
Processor                                  47779 non-null object
SmartScreen                                47779 non-null object
Census_MDC2FormFactor                      47779 non-null object
Platform                                   47779 non-null object
OsBuild                                    47779 non-null int64
Census_IsAlwaysOnAlwaysConnectedCapable    47779 non-null float64
Wdft_IsGamer                               47779 non-null float64
RtpStateBitfield                           47779 non-null float64
IsSxsPassiveMode                           47779 non-null int64
Census_HasOpticalDiskDrive                 47779 non-null int64
Census_IsVirtualDevice                     47779 non-null float64
HasDetections                              47

In [32]:
# Make a new dataframe that's the bitwise representation of the string
AVProduct_df=data_df['AVProductStatesIdentifier'].str.split(pat="",n=-1,expand=True)
AVProduct_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1,
1,,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1,
2,,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1,
3,,0,1,0,1,1,1,0,0,0,1,0,0,0,0,1,1,0,
4,,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1,


In [33]:
for n in range(1 , 18):
    data_df[f"AV_states_{n}"]=AVProduct_df[n].astype(str)

In [34]:
data_df=data_df.drop(["AVProductStatesIdentifier"],axis=1)

In [35]:
y=data_df["HasDetections"]
len(y)

47779

In [36]:
X=pd.get_dummies(data_df, drop_first=True)
X=X.drop(["HasDetections"],axis=1)
dummy_columns=X.columns.get_values()
dummy_columns=dummy_columns.tolist()

X.head()

,OsBuild,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,RtpStateBitfield,IsSxsPassiveMode,Census_HasOpticalDiskDrive,Census_IsVirtualDevice,EngineVersion_1.1.12101.0,EngineVersion_1.1.12603.0,EngineVersion_1.1.12805.0,...,AV_states_8_1,AV_states_9_1,AV_states_10_1,AV_states_11_1,AV_states_12_1,AV_states_13_1,AV_states_14_1,AV_states_15_1,AV_states_16_1,AV_states_17_1
0,15063,0.0,0.0,7.0,0,0,0.0,0,0,0,...,0,0,1,1,0,0,0,1,1,1
1,16299,0.0,1.0,7.0,0,0,0.0,0,0,0,...,0,0,1,1,0,0,0,1,1,1
2,15063,0.0,1.0,7.0,0,0,0.0,0,0,0,...,0,0,1,1,0,0,0,1,1,1
3,17134,0.0,0.0,7.0,0,0,0.0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
4,17134,0.0,0.0,7.0,0,0,0.0,0,0,0,...,0,0,1,1,0,0,0,1,1,1


In [37]:
dummy_columns

['OsBuild',
 'Census_IsAlwaysOnAlwaysConnectedCapable',
 'Wdft_IsGamer',
 'RtpStateBitfield',
 'IsSxsPassiveMode',
 'Census_HasOpticalDiskDrive',
 'Census_IsVirtualDevice',
 'EngineVersion_1.1.12101.0',
 'EngineVersion_1.1.12603.0',
 'EngineVersion_1.1.12805.0',
 'EngineVersion_1.1.12902.0',
 'EngineVersion_1.1.13000.0',
 'EngineVersion_1.1.13103.0',
 'EngineVersion_1.1.13202.0',
 'EngineVersion_1.1.13303.0',
 'EngineVersion_1.1.13407.0',
 'EngineVersion_1.1.13504.0',
 'EngineVersion_1.1.13601.0',
 'EngineVersion_1.1.13701.0',
 'EngineVersion_1.1.13704.0',
 'EngineVersion_1.1.13802.0',
 'EngineVersion_1.1.13803.0',
 'EngineVersion_1.1.13804.0',
 'EngineVersion_1.1.13903.0',
 'EngineVersion_1.1.14003.0',
 'EngineVersion_1.1.14104.0',
 'EngineVersion_1.1.14201.0',
 'EngineVersion_1.1.14202.0',
 'EngineVersion_1.1.14305.0',
 'EngineVersion_1.1.14306.0',
 'EngineVersion_1.1.14405.2',
 'EngineVersion_1.1.14500.5',
 'EngineVersion_1.1.14600.4',
 'EngineVersion_1.1.14700.3',
 'EngineVersion_1

In [38]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)

In [39]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)

In [40]:

print(f"Training: {rf.score(X_train, y_train)}")
print(f"Testing: {rf.score(X_test, y_test)}" )



Training: 0.7219400569291734
Testing: 0.6046044370029301


In [41]:
importances = rf.feature_importances_
sorted(zip(rf.feature_importances_, dummy_columns), reverse=True)

[(0.1814896622355767, 'OsBuild'),
 (0.09445401121300259, 'SmartScreen_ExistsNotSet'),
 (0.05267454345245631, 'Wdft_IsGamer'),
 (0.03409484092516747, 'Census_HasOpticalDiskDrive'),
 (0.034048240405541906, 'Census_MDC2FormFactor_Notebook'),
 (0.032978657427670276, 'SmartScreen_off'),
 (0.032005456718573216, 'SmartScreen_RequireAdmin'),
 (0.030120254905681345, 'EngineVersion_1.1.15100.1'),
 (0.026889231105273678, 'Census_MDC2FormFactor_Desktop'),
 (0.025955573808572677, 'EngineVersion_1.1.15200.1'),
 (0.017415892357585158, 'Census_MDC2FormFactor_Convertible'),
 (0.017027432970976225, 'AV_states_3_1'),
 (0.016998582364867943, 'AV_states_7_1'),
 (0.016274093939994287, 'AV_states_16_1'),
 (0.0159631266595338, 'AV_states_12_1'),
 (0.015243076365069285, 'AV_states_10_1'),
 (0.01511922026342876, 'AV_states_9_1'),
 (0.014560712319144247, 'AV_states_11_1'),
 (0.014434678894539234, 'Census_IsAlwaysOnAlwaysConnectedCapable'),
 (0.014237600453226092, 'AV_states_15_1'),
 (0.013954449608882728, 'AV_st

In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rf.predict(X_test),
                            ))

             precision    recall  f1-score   support

          0       0.62      0.53      0.57      5921
          1       0.59      0.68      0.63      6024

avg / total       0.61      0.60      0.60     11945



## Iteration 9

This iteration is going to go back to the one hot encoded Census_OSVersion and use the bitwise separation for AVProductStates


In [50]:
data_df=pd.read_csv(file)

data_df=data_df[["EngineVersion","AVProductStatesIdentifier","Processor","SmartScreen","Census_MDC2FormFactor","Census_OSVersion", 
                 "Census_IsAlwaysOnAlwaysConnectedCapable","Wdft_IsGamer",
                 "RtpStateBitfield", "IsSxsPassiveMode" , "Census_HasOpticalDiskDrive", "Census_IsVirtualDevice", "HasDetections"]]

data_df.head()


,EngineVersion,AVProductStatesIdentifier,Processor,SmartScreen,Census_MDC2FormFactor,Census_OSVersion,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,RtpStateBitfield,IsSxsPassiveMode,Census_HasOpticalDiskDrive,Census_IsVirtualDevice,HasDetections
0,1.1.15200.1,53447.0,x64,NaN,Desktop,10.0.15063.0,0.0,0.0,7.0,0,0,0.0,0
1,1.1.15200.1,53447.0,x64,RequireAdmin,Desktop,10.0.16299.125,0.0,1.0,7.0,0,0,0.0,0
2,1.1.15100.1,53447.0,x64,NaN,Notebook,10.0.15063.1088,0.0,1.0,7.0,0,0,0.0,1
3,1.1.14600.4,47238.0,x64,NaN,Notebook,10.0.17134.112,0.0,0.0,7.0,0,0,0.0,0
4,1.1.15200.1,53447.0,x64,NaN,Desktop,10.0.17134.254,0.0,0.0,7.0,0,0,0.0,1


In [51]:
data_df[["Census_IsAlwaysOnAlwaysConnectedCapable","Wdft_IsGamer", "RtpStateBitfield", "Census_IsVirtualDevice"]].fillna(value=0, inplace=True)
data_df["SmartScreen"].fillna(value="off", inplace=True)
data_df["SmartScreen"].replace("Off", "off", inplace=True)
data_df["SmartScreen"].replace("On", "on", inplace=True)
data_df["SmartScreen"].fillna(value="off", inplace=True)
data_df["AVProductStatesIdentifier"].fillna(value="0", inplace=True)

## This drops any remaining nulls 
data_df.dropna(how='any',inplace=True)
y=data_df["HasDetections"]
len(y)

47779

In [52]:
# Convert the float in the AVProductStatesIdentifier feature to a binary
data_df['AVProductStatesIdentifier'] = data_df.AVProductStatesIdentifier.apply(lambda x: format(int(x), '04b'))
# Convert the Binary to a string
data_df['AVProductStatesIdentifier']=data_df['AVProductStatesIdentifier'].astype(str)
# Add Leading 0's to the sting
data_df["AVProductStatesIdentifier"]=data_df["AVProductStatesIdentifier"].str.zfill(17)
# Make a new dataframe that's the bitwise representation of the string
AVProduct_df=data_df['AVProductStatesIdentifier'].str.split(pat="",n=-1,expand=True)
AVProduct_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1,
1,,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1,
2,,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1,
3,,0,1,0,1,1,1,0,0,0,1,0,0,0,0,1,1,0,
4,,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1,


In [53]:
# Add the individual columns to the dataframe
for n in range(1 , 18):
    data_df[f"AV_states_{n}"]=AVProduct_df[n].astype(str)
    
# Drop the original column
data_df=data_df.drop(["AVProductStatesIdentifier"],axis=1)
data_df.head()

,EngineVersion,Processor,SmartScreen,Census_MDC2FormFactor,Census_OSVersion,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,RtpStateBitfield,IsSxsPassiveMode,Census_HasOpticalDiskDrive,...,AV_states_8,AV_states_9,AV_states_10,AV_states_11,AV_states_12,AV_states_13,AV_states_14,AV_states_15,AV_states_16,AV_states_17
0,1.1.15200.1,x64,off,Desktop,10.0.15063.0,0.0,0.0,7.0,0,0,...,0,0,1,1,0,0,0,1,1,1
1,1.1.15200.1,x64,RequireAdmin,Desktop,10.0.16299.125,0.0,1.0,7.0,0,0,...,0,0,1,1,0,0,0,1,1,1
2,1.1.15100.1,x64,off,Notebook,10.0.15063.1088,0.0,1.0,7.0,0,0,...,0,0,1,1,0,0,0,1,1,1
3,1.1.14600.4,x64,off,Notebook,10.0.17134.112,0.0,0.0,7.0,0,0,...,0,0,1,0,0,0,0,1,1,0
4,1.1.15200.1,x64,off,Desktop,10.0.17134.254,0.0,0.0,7.0,0,0,...,0,0,1,1,0,0,0,1,1,1


In [55]:
data_df=data_df.drop(["HasDetections"],axis=1)
X=pd.get_dummies(data_df, drop_first=True)
dummy_columns=X.columns.get_values()
dummy_columns=dummy_columns.tolist()

X.head()

,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,RtpStateBitfield,IsSxsPassiveMode,Census_HasOpticalDiskDrive,Census_IsVirtualDevice,EngineVersion_1.1.12101.0,EngineVersion_1.1.12603.0,EngineVersion_1.1.12805.0,EngineVersion_1.1.12902.0,...,AV_states_8_1,AV_states_9_1,AV_states_10_1,AV_states_11_1,AV_states_12_1,AV_states_13_1,AV_states_14_1,AV_states_15_1,AV_states_16_1,AV_states_17_1
0,0.0,0.0,7.0,0,0,0.0,0,0,0,0,...,0,0,1,1,0,0,0,1,1,1
1,0.0,1.0,7.0,0,0,0.0,0,0,0,0,...,0,0,1,1,0,0,0,1,1,1
2,0.0,1.0,7.0,0,0,0.0,0,0,0,0,...,0,0,1,1,0,0,0,1,1,1
3,0.0,0.0,7.0,0,0,0.0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
4,0.0,0.0,7.0,0,0,0.0,0,0,0,0,...,0,0,1,1,0,0,0,1,1,1


In [56]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)

In [57]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)

In [58]:

print(f"Training: {rf.score(X_train, y_train)}")
print(f"Testing: {rf.score(X_test, y_test)}" )

Training: 0.7906178489702517
Testing: 0.606613645876936


In [59]:
for i  in range(len(dummy_columns)):
    dummy_columns[i]=str(dummy_columns[i])

In [60]:
importances = rf.feature_importances_
sorted(zip(rf.feature_importances_, dummy_columns), reverse=True)

[(0.0658759711212522, 'SmartScreen_ExistsNotSet'),
 (0.04135938185043289, 'Wdft_IsGamer'),
 (0.03117724206016056, 'Census_MDC2FormFactor_Notebook'),
 (0.027096110226989434, 'SmartScreen_RequireAdmin'),
 (0.026933578343838364, 'Census_HasOpticalDiskDrive'),
 (0.02481198777513938, 'Census_MDC2FormFactor_Desktop'),
 (0.024704537911997934, 'SmartScreen_off'),
 (0.022320346796909297, 'EngineVersion_1.1.15100.1'),
 (0.020346109789616693, 'EngineVersion_1.1.15200.1'),
 (0.0164124712356143, 'Census_OSVersion_10.0.17134.228'),
 (0.016321546097426035, 'Census_OSVersion_10.0.16299.431'),
 (0.0149251103681371, 'Census_OSVersion_10.0.17134.165'),
 (0.014733734548469941, 'Census_MDC2FormFactor_Convertible'),
 (0.012482022916901224, 'Census_OSVersion_10.0.17134.112'),
 (0.011744573845688328, 'Census_OSVersion_10.0.16299.371'),
 (0.0115376411899571, 'Census_OSVersion_10.0.16299.547'),
 (0.011231402822364405, 'AV_states_9_1'),
 (0.011069500211609316, 'AV_states_16_1'),
 (0.010994016855917714, 'Census_I

In [61]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rf.predict(X_test),
                            ))

             precision    recall  f1-score   support

          0       0.61      0.59      0.60      5921
          1       0.61      0.63      0.62      6024

avg / total       0.61      0.61      0.61     11945



# Iteration 10

Use Iteration, 9 but substitute Census_MDC2FormFactor for  Census_TotalPhysicalRAM
and Census_ProcessorCoreCount

Also, go back to the one hot encoded Census_OSVersion instead of platform and build.

Will keep the bit encoded AVProductStatesIdentifier

In [70]:
data_df=pd.read_csv(file)
data_df=data_df[["EngineVersion","AVProductStatesIdentifier","Processor","SmartScreen","Census_TotalPhysicalRAM", "Census_ProcessorCoreCount","Census_OSVersion", 
                 "Census_IsAlwaysOnAlwaysConnectedCapable","Wdft_IsGamer",
                 "RtpStateBitfield", "IsSxsPassiveMode" , "Census_HasOpticalDiskDrive", "Census_IsVirtualDevice", "HasDetections"]]

data_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
EngineVersion                              50000 non-null object
AVProductStatesIdentifier                  49770 non-null float64
Processor                                  50000 non-null object
SmartScreen                                32108 non-null object
Census_TotalPhysicalRAM                    49546 non-null float64
Census_ProcessorCoreCount                  49763 non-null float64
Census_OSVersion                           50000 non-null object
Census_IsAlwaysOnAlwaysConnectedCapable    49613 non-null float64
Wdft_IsGamer                               48271 non-null float64
RtpStateBitfield                           49816 non-null float64
IsSxsPassiveMode                           50000 non-null int64
Census_HasOpticalDiskDrive                 50000 non-null int64
Census_IsVirtualDevice                     49927 non-null float64
HasDetections                            

In [71]:
data_df[["Census_IsAlwaysOnAlwaysConnectedCapable","Wdft_IsGamer", "RtpStateBitfield", "Census_IsVirtualDevice"]].fillna(value=0, inplace=True)
data_df["SmartScreen"].fillna(value="off", inplace=True)
data_df["SmartScreen"].replace("Off", "off", inplace=True)
data_df["SmartScreen"].replace("On", "on", inplace=True)
data_df["SmartScreen"].fillna(value="off", inplace=True)
data_df["AVProductStatesIdentifier"].fillna(value="0", inplace=True)

## This drops any remaining nulls 
data_df.dropna(how='any',inplace=True)
y=data_df["HasDetections"]
len(y)

47578

In [72]:
# Convert the float in the AVProductStatesIdentifier feature to a binary
data_df['AVProductStatesIdentifier'] = data_df.AVProductStatesIdentifier.apply(lambda x: format(int(x), '04b'))
# Convert the Binary to a string
data_df['AVProductStatesIdentifier']=data_df['AVProductStatesIdentifier'].astype(str)
# Add Leading 0's to the sting
data_df["AVProductStatesIdentifier"]=data_df["AVProductStatesIdentifier"].str.zfill(17)
data_df.head()

,EngineVersion,AVProductStatesIdentifier,Processor,SmartScreen,Census_TotalPhysicalRAM,Census_ProcessorCoreCount,Census_OSVersion,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,RtpStateBitfield,IsSxsPassiveMode,Census_HasOpticalDiskDrive,Census_IsVirtualDevice,HasDetections
0,1.1.15200.1,01101000011000111,x64,off,4096.0,2.0,10.0.15063.0,0.0,0.0,7.0,0,0,0.0,0
1,1.1.15200.1,01101000011000111,x64,RequireAdmin,4096.0,4.0,10.0.16299.125,0.0,1.0,7.0,0,0,0.0,0
2,1.1.15100.1,01101000011000111,x64,off,4096.0,2.0,10.0.15063.1088,0.0,1.0,7.0,0,0,0.0,1
3,1.1.14600.4,01011100010000110,x64,off,4096.0,4.0,10.0.17134.112,0.0,0.0,7.0,0,0,0.0,0
4,1.1.15200.1,01101000011000111,x64,off,32768.0,12.0,10.0.17134.254,0.0,0.0,7.0,0,0,0.0,1


In [73]:
for n in range(1 , 18):
    data_df[f"AV_states_{n}"]=AVProduct_df[n].astype(str)
    
data_df=data_df.drop(["AVProductStatesIdentifier"],axis=1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47578 entries, 0 to 49999
Data columns (total 14 columns):
EngineVersion                              47578 non-null object
AVProductStatesIdentifier                  47578 non-null object
Processor                                  47578 non-null object
SmartScreen                                47578 non-null object
Census_TotalPhysicalRAM                    47578 non-null float64
Census_ProcessorCoreCount                  47578 non-null float64
Census_OSVersion                           47578 non-null object
Census_IsAlwaysOnAlwaysConnectedCapable    47578 non-null float64
Wdft_IsGamer                               47578 non-null float64
RtpStateBitfield                           47578 non-null float64
IsSxsPassiveMode                           47578 non-null int64
Census_HasOpticalDiskDrive                 47578 non-null int64
Census_IsVirtualDevice                     47578 non-null float64
HasDetections                             

In [74]:
data_df=data_df.drop(["HasDetections"],axis=1)
X=pd.get_dummies(data_df, drop_first=True)
dummy_columns=X.columns.get_values()
dummy_columns=dummy_columns.tolist()

X.head()

,Census_TotalPhysicalRAM,Census_ProcessorCoreCount,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,RtpStateBitfield,IsSxsPassiveMode,Census_HasOpticalDiskDrive,Census_IsVirtualDevice,EngineVersion_1.1.12603.0,EngineVersion_1.1.12805.0,...,AV_states_8_1,AV_states_9_1,AV_states_10_1,AV_states_11_1,AV_states_12_1,AV_states_13_1,AV_states_14_1,AV_states_15_1,AV_states_16_1,AV_states_17_1
0,4096.0,2.0,0.0,0.0,7.0,0,0,0.0,0,0,...,0,0,1,1,0,0,0,1,1,1
1,4096.0,4.0,0.0,1.0,7.0,0,0,0.0,0,0,...,0,0,1,1,0,0,0,1,1,1
2,4096.0,2.0,0.0,1.0,7.0,0,0,0.0,0,0,...,0,0,1,1,0,0,0,1,1,1
3,4096.0,4.0,0.0,0.0,7.0,0,0,0.0,0,0,...,0,0,1,0,0,0,0,1,1,0
4,32768.0,12.0,0.0,0.0,7.0,0,0,0.0,0,0,...,0,0,1,1,0,0,0,1,1,1


In [75]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)

In [76]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)

In [77]:

print(f"Training: {rf.score(X_train, y_train)}")
print(f"Testing: {rf.score(X_test, y_test)}" )

Training: 0.8350475016114116
Testing: 0.6051282051282051


In [78]:
for i  in range(len(dummy_columns)):
    dummy_columns[i]=str(dummy_columns[i])

In [79]:
importances = rf.feature_importances_
sorted(zip(rf.feature_importances_, dummy_columns), reverse=True)

[(0.13632049097901255, 'Census_TotalPhysicalRAM'),
 (0.07827239780763401, 'Census_ProcessorCoreCount'),
 (0.05446638533338434, 'SmartScreen_ExistsNotSet'),
 (0.03422508033803156, 'Wdft_IsGamer'),
 (0.023694168190560482, 'Census_HasOpticalDiskDrive'),
 (0.021806705633699415, 'SmartScreen_RequireAdmin'),
 (0.021768188960851993, 'SmartScreen_off'),
 (0.019535830994793363, 'EngineVersion_1.1.15100.1'),
 (0.018299385151660408, 'EngineVersion_1.1.15200.1'),
 (0.014965217025030566, 'Census_OSVersion_10.0.16299.431'),
 (0.01451215466853889, 'Census_OSVersion_10.0.17134.228'),
 (0.012531872494073807, 'Census_IsAlwaysOnAlwaysConnectedCapable'),
 (0.012442311019542416, 'Census_OSVersion_10.0.17134.165'),
 (0.01111884726610157, 'Census_OSVersion_10.0.17134.112'),
 (0.010486828487987265, 'Census_OSVersion_10.0.16299.547'),
 (0.01044725644889339, 'Census_OSVersion_10.0.16299.371'),
 (0.00984494384306846, 'AV_states_10_1'),
 (0.009326694179535492, 'AV_states_16_1'),
 (0.009179560734462243, 'AV_states

In [80]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rf.predict(X_test),
                            ))

             precision    recall  f1-score   support

          0       0.60      0.60      0.60      5888
          1       0.61      0.61      0.61      6007

avg / total       0.61      0.61      0.61     11895

